In [1]:
import random as rm
import copy
from PIL import Image, ImageDraw
import numpy as np

In [2]:
def zliczanie_czarnych(populacja,liczba_osobnikow):
    eval=[]
    for i in range(liczba_osobnikow):
        im = Image.open("./gen_2.png")
        obr=ImageDraw.Draw(im)
        for j in range(dl_genu):
            obr.ellipse([populacja[i][j][0]-10, populacja[i][j][1]-10, populacja[i][j][0]+10, populacja[i][j][1]+10], fill='red', outline='red')
        im.load()
        #im.show()
        k = int(im.getcolors()[-1][0])
        im.close()
        eval += [k]
    return eval

#ev = zliczanie_czarnych(pop,n)
#print(ev)

In [3]:
def krzyzowanie(p1, p2, sr, r):
    wew_os1 = [x for x in p1 if abs(x[0] - sr[0] + x[1] - sr[1]) <= r]
    zew_os1 = [x for x in p1 if abs(x[0] - sr[0] + x[1] - sr[1]) >= r]
    #print('wew_os1: ', wew_os1, ' zew_os1: ',zew_os1)
    wew_os2 = [x for x in p2 if abs(x[0] - sr[0] + x[1] - sr[1]) <= r]
    zew_os2 = [x for x in p2 if abs(x[0] - sr[0] + x[1] - sr[1]) >= r]
    #print('wew_os2: ', wew_os2, ' zew_os2: ',zew_os2)
    
    nowy1 = wew_os1 + zew_os2
    nowy2 = wew_os2 + zew_os1
    
    
    if(len(nowy1) > len(nowy2)):
        while len(nowy1) > dl_genu:
            del nowy1[rm.randint(0,dl_genu-1)]

        while len(nowy2) < dl_genu:
            i = rm.randint(0, len(wew_os1) - 1)
            if not wew_os1[i] in nowy2:
                nowy2 += [wew_os1[i]]
                
    else:
        while len(nowy2) > dl_genu:
            del nowy2[rm.randint(0,dl_genu-1)]

        while len(nowy1) < dl_genu:
            i = rm.randint(0, len(wew_os2) - 1)
            if not wew_os2[i] in nowy1:
                nowy1 += [wew_os2[i]]

    return [nowy1,nowy2]

In [4]:
def krzyzowanie_linia(p1, p2, szer_mapy, wys_mapy):
    poczatek_linii = rm.randint(0,wys_mapy)
    koniec_linii = rm.randint(0,wys_mapy)
    b = poczatek_linii
    a = (-koniec_linii+b)/szer_mapy
    pod_linia1 = [x for x in p1 if abs(x[1]) <= (a*x[0]+b)] 
    #Do listy wew_os1 dołączamy gen (parę współrzędnych) z osobnika p1 jeżeli wsp y jest mniejsza niż wartość 
    #funkcji liniowej dla danego x (czyli sklep leży pod linią) - list ta zatem może maksymalnie liczyć 12 genów
    nad_linia1 = [x for x in p1 if abs(x[1]) >= (a*x[0]+b)] 
    #print('pod_linia1: ', pod_linia1, ' nad_linia1: ',nad_linia1)
    pod_linia2 = [x for x in p2 if abs(x[1]) <= (a*x[0]+b)] 
    nad_linia2 = [x for x in p2 if abs(x[1]) >= (a*x[0]+b)] 
    nowy1 = pod_linia1 + nad_linia1 #lista nowy1 może maksymalnie liczyć 24 geny
    nowy2 = pod_linia2 + nad_linia2
    
    
    if(len(nowy1) > len(nowy2)): #Jeżeli lista nowy 1 jest dłuższa to dopóki będzie dłuższa niż przewidywana długość genu,
        #należy usuwać z niej losowe geny
        while len(nowy1) > dl_genu:
            del nowy1[rm.randint(0,dl_genu-1)]

        while len(nowy2) < dl_genu:
            i = rm.randint(0, len(wew_os1) - 1)
            if not wew_os1[i] in nowy2:
                nowy2 += [wew_os1[i]]
                
    else:
        while len(nowy2) > dl_genu:
            del nowy2[rm.randint(0,dl_genu-1)]

        while len(nowy1) < dl_genu:
            i = rm.randint(0, len(wew_os2) - 1)
            if not wew_os2[i] in nowy1:
                nowy1 += [wew_os2[i]]
    
    return [nowy1,nowy2]

In [5]:
def odleglosc_euklidesowa_Nwymiarowa(x,y):
    suma=0
    for i in range(len(x)):
        suma = (x[i]-y[i])**2 +suma
    return np.sqrt(suma)

In [6]:
def krzyzowanie_odleglosc(p1, p2, szer_mapy, wys_mapy):
    pkt_centralny = [szer_mapy/2,wys_mapy/2]
    p1_odleglosci = [] #Obliczam odległości sklepów od środka mapy - osobnik 1
    for x in p1:
        p1_odleglosci.append(odleglosc_euklidesowa_Nwymiarowa(x,pkt_centralny))
    p2_odleglosci = [] #Obliczam odległości sklepów od środka mapy - osobnik 2
    for x in p2:
        p2_odleglosci.append(odleglosc_euklidesowa_Nwymiarowa(x,pkt_centralny))
    nowy1 = []
    nowy2 = []
    for i in range(7): #pierwsza część listy z odległościami - minima
        #do pierwszego dziecka dodaję gen o najmniejszej odległości od środka - z listy genów I osobnika
        nowy1.append(p1[p1_odleglosci.index(min(p1_odleglosci))])
        p1_odleglosci.remove(min(p1_odleglosci)) #usuwam tę minimalnąwartość
        #Do drugiego dziecka dodaję gen o najmniejszej odległości od środka - z listy genów II osobnika
        nowy2.append(p2[p2_odleglosci.index(min(p2_odleglosci))])
        p2_odleglosci.remove(min(p2_odleglosci))#usuwam tę minimalną wartość
    while(len(p1_odleglosci)>0):  #druga część listy z odległościami - maxima  
        #do pierwszego dziecka dodaję gen o największej odległości od środka - z listy genów II osobnika
        nowy1.append(p2[p2_odleglosci.index(max(p2_odleglosci))])
        p2_odleglosci.remove(min(p2_odleglosci)) #usuwam tę maksymalną wartość
        #do drugiego osobnika dodaję gen o największej odległości od środka - z listy genów I osobnika
        nowy2.append(p1[p1_odleglosci.index(max(p1_odleglosci))])
        p1_odleglosci.remove(min(p1_odleglosci))
    return [nowy1,nowy2]

In [7]:
#prawd_mutacji = 0.5
#p_mut = random.random()
def mutacja(child):
    polozenie_do_wymiany = rm.randint(0,dl_genu-1)
    losowe_polozenie = [rm.randint(0,szer_mapy), rm.randint(0,wys_mapy)]
    child[polozenie_do_wymiany] = losowe_polozenie
    
    return child

In [8]:
def mutacja_przesun(child):
    polozenie_do_wymiany = rm.randint(0,dl_genu-1)
    
    #wektor przesunięcia losowany jest z zakresu (-128, 128) - przesunięcie może 
    #następować w każdym kierunku
    wektor_przesuniecia = [rm.randint(-szer_mapy,szer_mapy), rm.randint(-wys_mapy,wys_mapy)]

    child[polozenie_do_wymiany][0] += wektor_przesuniecia[0]
    child[polozenie_do_wymiany][1] += wektor_przesuniecia[1]
    
    
    #Lewy górny róg obrazka ma wsp. = (0,0), a prawy dolny = (128,128), więc gdy po przesunięciu
    #o losowy wektor któraś ze współrzędnych wyjdzie poza zakres trzeba ją sprowadzić do wartości
    #z przedziału (0,128)
    if(child[polozenie_do_wymiany][0] < 0):
        while(child[polozenie_do_wymiany][0] < 0):
            child[polozenie_do_wymiany][0] += rm.randint(0,szer_mapy)
    elif(child[polozenie_do_wymiany][0] > szer_mapy):
        while(child[polozenie_do_wymiany][0] > szer_mapy):
            child[polozenie_do_wymiany][0] -= rm.randint(0,szer_mapy)
            
    if(child[polozenie_do_wymiany][1] < 0):
        while(child[polozenie_do_wymiany][1] < 0):
            child[polozenie_do_wymiany][1] += rm.randint(0,wys_mapy)
    elif(child[polozenie_do_wymiany][1] > szer_mapy):
        while(child[polozenie_do_wymiany][1] > szer_mapy):
            child[polozenie_do_wymiany][1] -= rm.randint(0,wys_mapy)
    
    return child

In [44]:
#Mutacja polega na zamianie miejscami współrzędnych danego położenia - wsp. x przyjmuje wartość wsp y,
#a wsp y wartość wsp x 
def mutacja_zamien_wsp(child):
    #losowanie, które położenie ulegnie mutacji
    polozenie_do_wymiany = rm.randint(0,dl_genu-1)
    
    #Zamiana miejscami współrzędnych
    tmp = child[polozenie_do_wymiany][0]
    child[polozenie_do_wymiany][0] = child[polozenie_do_wymiany][1]
    child[polozenie_do_wymiany][1] = tmp
    
    return child

In [10]:
def wybierz_x_najlepszych(populacja,x,n):
    popul = copy.deepcopy(populacja)
    ev = zliczanie_czarnych(popul,n)
    najlepsi = []
    for os in range(x):
        index = ev.index(min(ev))
        najlepsi.append(popul[index])
        ev.remove(min(ev))
        popul.remove(popul[index])
    return najlepsi

In [25]:
def selekcja_turniej(populacja, ev):
    rodzice = []
    for j in range(2):
        #Wylosowanie dwóch rodziców
        parent1 = rm.randint(0, len(populacja)-1)
        parent2 = rm.randint(0, len(populacja)-1)
        os = populacja[parent1] if ev[parent1] <= ev[parent2] else populacja[parent2] #wybór zwycięskiego osobnika
        rodzice += [os] #dodanie zwycięskiego osobnika do listy rodziców
    return rodzice
        

In [12]:
def selekcja_ruletka(populacja, ev):
    rodzice = []
    minimum = min(ev)
    ev = [i+(minimum + 1) for i in ev]
    ev_suma = sum(ev)
    ev = [i/ev_suma for i in ev] #prawdopodobieństwo wylosowania danego rodzica
    dystrybuanta = [sum(ev[:i+1]) for i in range(len(ev))]

    for i in range(2):
            prawd = rm.random()
            if(prawd < dystrybuanta[0]):
                os = pop[0]
                rodzice += [os] 
            else:
                for j in range(1,len(dystrybuanta)):
                    if(prawd > dystrybuanta[j-1] and prawd <= dystrybuanta[j]):
                        os = pop[j]
                        rodzice += [os] 
    return rodzice

In [13]:
def compute_sel_prob(ev):
    n = len(ev)
    rank_sum = n * (n + 1) / 2
    for rank, ind_fitness in enumerate(sorted(ev), 1):
        yield rank, ind_fitness, float(rank) / rank_sum


In [14]:
def selekcja_ranking(populacja, ev):
    rodzice = []
    sel_probs = []
    ind_fitnesses = []
    for rank, ind_fitness, sel_prob in compute_sel_prob(ev):
        sel_probs.append(sel_prob)
        ind_fitnesses.append(ind_fitness)
    dystrybuanta = [sum(sel_probs[:i+1]) for i in range(len(sel_probs))]
    for i in range(2):
            prawd = rm.random()
            if(prawd < dystrybuanta[0]):
                index = ev.index(ind_fitnesses[0])
                os = pop[index]
                rodzice += [os] 
            else:
                for j in range(1,len(dystrybuanta)):
                    if(prawd > dystrybuanta[j-1] and prawd <= dystrybuanta[j]):
                        index = ev.index(ind_fitnesses[0])
                        os = pop[index]
                        rodzice += [os] 
    return rodzice

In [42]:
def petla_glowna(liczba_generacji,pop,ev,wersja_selekcji,wersja_krzyzowania,wersja_mutacji):
    for generacja in range(liczba_generacji):
            children = []
            for i in range(12):
                if wersja_selekcji==1:
                    parents = selekcja_turniej(pop,ev)
                elif wersja_selekcji==2:    
                    parents = selekcja_ruletka(pop,ev)
                else:
                    parents = selekcja_ranking(pop,ev)
                #Krzyżowanie lub bez krzyżowania
                #prawd_krzyżow = 0.5
                prawd_krzyżow = 0.9
                p_krzyż = rm.random()
                if p_krzyż < prawd_krzyżow:
                    if wersja_krzyzowania==1:
                        ch = krzyzowanie(parents[0], parents[1], s, R)     
                        #Krzyżowanie z wykorzystaniem okręgu 
                    elif wersja_krzyzowania == 2:
                        ch = krzyzowanie_linia(parents[0], parents[1],szer_mapy,wys_mapy)     
                        #Krzyżowanie z wykorzystaniem linii
                    else:
                        ch = krzyzowanie_odleglosc(parents[0], parents[1],szer_mapy,wys_mapy)     
                        #Krzyżowanie z wykorzystaniem odległości od środka
                    children.append(ch[0])
                    children.append(ch[1])
                else:
                    children.append(parents[0])
                    children.append(parents[1])

            #Mutacja
            prawd_mutacji = 0.5
            #prawd_mutacji = 0.1
            for i in range(12):
                p_mut = rm.random()
                if p_mut < prawd_mutacji:
                    if wersja_mutacji ==1:
                        children[i] = mutacja(children[i])
                    elif wersja_mutacji ==2:
                        children[i] = mutacja_przesun(children[i])  
                    else:
                        children[i] = mutacja_zamien_wsp(children[i])

            #Tworzenie nowej populacji z rodziców oraz dzieci
            pop = pop + children

            #Wybór 30 najlepszych osobników jako nowych rodziców
            nowa_pop = wybierz_x_najlepszych(pop,30,len(pop))
            pop = nowa_pop
            n = len(pop)
            ev = zliczanie_czarnych(pop,n)
            #Wybór najlepszego osobnika i zapisanie jego układu do pliku
            najlepszy_os = wybierz_x_najlepszych(pop,1,n)
    return najlepszy_os

In [48]:
#Algorytm genetyczny

#Ustawienia początkowe
n = 44
dl_genu = 15
szer_mapy = 128
wys_mapy = 128

im = Image.open("./gen_1.png")
im.save("./gen_2.png")
im.close
im = Image.open("./gen_2.png")

s = [rm.randint(0,szer_mapy), rm.randint(0,wys_mapy)]
print('środek: ',s)
R = rm.randint(0,wys_mapy)
print('promień: ',R)

#Utworzenie populacji początkowej
pop = []
rm.seed(72)
for i in range(n):
    cele = [[rm.randint(0,szer_mapy), rm.randint(0,wys_mapy)] for j in range(0,dl_genu)]
    pop+=[cele]
     
ev = zliczanie_czarnych(pop,n)

#Główna pętla
liczba_generacji = 100 #100 #200 #500
wersja_selekcji = 1
wersja_krzyzowania = 1
wersja_mutacji = 2
najlepszy_os = petla_glowna(liczba_generacji,pop,ev,wersja_selekcji, wersja_krzyzowania, wersja_mutacji)
print(zliczanie_czarnych(najlepszy_os,1))

im = Image.open("./gen_2.png")
obr = ImageDraw.Draw(im)
for j in range(dl_genu):
    obr.ellipse([najlepszy_os[0][j][0]-10, najlepszy_os[0][j][1]-10, najlepszy_os[0][j][0]+10, najlepszy_os[0][j][1]+10], fill='red', outline='red')

im.save("./najlepszy502.png")

    


środek:  [27, 34]
promień:  104
[2841]
